In [1]:
%%writefile header.h

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <string.h>

int isspace(int argument);

typedef struct {
	char version[3]; 
	int width;
	int height;
	int maxGrayLevel;
	int *imageData;
	int *gx;
	int *gy;
} pgm;

void init_out_image( pgm* out, pgm image){
	int i, j;
	strcpy(out->version, image.version);
	out->width = image.width;
	out->height = image.height;
	out->maxGrayLevel = image.maxGrayLevel;
	
	out->imageData = (int*) malloc(out->height * out->width * sizeof(int));

	
	out->gx = (int*) malloc(out->height * out->width *  sizeof(int));
	
	
	out->gy = (int*) malloc(out->height * out->width * sizeof(int));

	
	for(i = 0; i < out->height * out->width; i++) {
			out->imageData[i] = image.imageData[i];
			out->gx[i] = image.imageData[i];
			out->gy[i] = image.imageData[i];
		}
}

void read_comments(FILE *input_image) {
	char ch;
	char line[100];

	while ((ch = fgetc(input_image)) != EOF && (isspace(ch)))  {
		;
    }
	if (ch == '#') {
        fgets(line, sizeof(line), input_image);
    } 
	else {
		fseek(input_image, -2L, SEEK_CUR);
	}
}

void read_pgm_file(char* dir, pgm* image) {
	FILE* input_image; 
	int i, j, num;

	input_image = fopen(dir, "rb");
	if (input_image == NULL) {
		printf("File could not opened!");
		return;
	} 
	
	fgets(image->version, sizeof(image->version), input_image);
	read_comments(input_image);

	fscanf(input_image, "%d %d %d", &image->width, &image->height, &image->maxGrayLevel);
	
	image->imageData = (int*) malloc(image->height * image->width *  sizeof(int));
	
	if (strcmp(image->version, "P2") == 0) {
		for (i = 0; i < image->height * image->width; i++) {
				fscanf(input_image, "%d", &num);
				image->imageData[i] = num;
			
		}	
	}
	else if (strcmp(image->version, "P5") == 0) {
		char *buffer;
		int buffer_size = image->height * image->width;
		buffer = (char*) malloc( ( buffer_size + 1) * sizeof(char));
		
		if(buffer == NULL) {
			printf("Can not allocate memory for buffer! \n");
			return;
		}
		fread(buffer, sizeof(char), image->width * image-> height, input_image);
		for (i = 0; i < image->height * image ->width; i++) {
			image->imageData[i] = buffer[i];
		}
		free(buffer);
	}
	fclose(input_image);
	//printf("_______________IMAGE INFO__________________\n");
	//printf("Version: %s \nWidth: %d \nHeight: %d \nMaximum Gray Level: %d \n", image->version, image->width, image->height, image->maxGrayLevel);
}

void write_pgm_file(pgm* image, char dir[], int*matrix) {
    FILE* out_image;
    int i, j, count = 0;
    out_image = fopen(dir, "wb");
    fprintf(out_image, "%s\n", image->version);

    if (strcmp(image->version, "P2") == 0) {
        fprintf(out_image, "%d %d\n", image->width, image->height);
        fprintf(out_image, "%d\n", image->maxGrayLevel);        
        count =0;
        for(i = 0; i < image->height * image->width; i++) {
            fprintf(out_image,"%4d", matrix[i]);
            count++;
            if(count % image->width ==0) fprintf(out_image,"\n");
        }
        
        
    }
    else if (strcmp(image->version, "P5") == 0) {
        for(i = 0; i < image->height * image->width; i++) {
                char num = image->imageData[i];
                fprintf(out_image,"%c ", num);
            } 
    } 
    fclose(out_image);
}


int convolution(pgm image, int kernel[3][3], int row, int col) {
	int sum = 0;
	sum = image.imageData[(row-1)*image.width+(col-1)] * kernel[0][0] +
				image.imageData[(row-1)*image.width  + col] * kernel[0][1] +
				image.imageData[(row-1)*image.width + (col+1)] * kernel[0][2] +
				image.imageData[row*image.width +(col-1)] * kernel[1][0] +
				image.imageData[row*image.width + col] * kernel[1][1] +
				image.imageData[row*image.width + (col+1)] * kernel[1][2] +
				image.imageData[(row+1)*image.width + (col-1)] * kernel[2][0] +
				image.imageData[(row+1)*image.width + col] * kernel[2][1] +
				image.imageData[(row+1)*image.width + (col+1)] * kernel[2][2];
	
	return sum;
}



void sobel_edge_detector(pgm image, pgm* out_image) {
	int i, j, k, gx, gy;
	int mx[3][3] = {
		{-1, 0, 1},
		{-2, 0, 2},
		{-1, 0, 1}
	};
	int my[3][3] = {
		{-1, -2, -1},
		{0, 0, 0},
		{1, 2, 1}
	};
	k = image.width;
	for (i = 1; i < (image.height - 1); i++) {
		for (j = 1; j < (image.width - 1); j++) {
      if (j==1)k++;
			gx = convolution(image, mx, i, j);
			gy = convolution(image, my, i, j);
			out_image->imageData[k] = sqrt(gx*gx + gy*gy);
			out_image->gx[k] = gx;
			out_image->gy[k] = gy;
      k++;
		}
    k++;
	}
	
}

Writing header.h


In [2]:
%%writefile pgm.c

#include "header.h"

int main(int argc, char **argv)
{
	pgm image, out_image;
	char *input_dir, *output_dir;
  input_dir = argv[1];
  output_dir = argv[2];
  int i,j;
	
	read_pgm_file(input_dir, &image);
  init_out_image(&out_image, image);
  sobel_edge_detector(image, &out_image);
 
  write_pgm_file(&out_image,output_dir, out_image.imageData);

  return 0;
}

Writing pgm.c


In [3]:
%%script bash
gcc pgm.c -o pgm -lm
./pgm test.pgm test_sobel.pgm

File could not opened!

In [4]:
%%writefile ppm.c

#include <stdlib.h>
#include <stdio.h>
#include <math.h>

//Estructura para guardar los valores de colores

void generaImagen_salida(int *img_rojo,int* img_verde,int *img_azul, int m, int n, int protocolo, char numero, char formato, char *nombre){
 	int r, g, b;
  FILE *newImagePointer;

	newImagePointer = fopen(nombre, "w");
	fprintf(newImagePointer, "%c%c \n%i %i\n%i\n", formato, numero, m, n, protocolo);

	for (int i = 0; i < m * n; i++)	
		fprintf(newImagePointer, "%i %i %i\n", img_rojo[i], img_verde[i], img_azul[i]);
	
	fclose(newImagePointer);
}

int convolucion(int *color, int m, int n, int *Gx, int *Gy, int *imgFinal){
  int kernel = 3;
  int i,j;
  int aux = 0;
  int auxC = 0;
  int multGx = 0, multGy=0;
  int sumNumGx = 0, sumNumGy = 0;
  int movDer=0, auxMovDer=0, movBajo;
  int pakBaje=0;
  int G = 0;
  int count=0;
  for(movBajo=0;abs(movBajo - n)>=kernel; movBajo++){
    for(movDer=0; abs(movDer - m)>=kernel; movDer++, count++){
      for(i=0; i < kernel; i++,aux+=m, auxC+=3){
        for(j=0; j < kernel; j++){
            //printf("%d ",color[aux+j+movDer + pakBaje]);
            multGx = color[aux+j+movDer + pakBaje] * Gx[auxC+j];
            multGy = color[aux+j+movDer + pakBaje] * Gy[auxC+j];
            sumNumGx = sumNumGx + multGx;
            sumNumGy = sumNumGy + multGy;
        }
        //printf("\n");
      }
      
      G = sqrt((pow((double)sumNumGx,2)) + (pow((double)sumNumGy,2)));
      if (G > 255) G = 255;
      //printf("sumNumGx = %d \n", sumNumGx);
      //printf("sumNumGy = %d \n", sumNumGy);
      //printf("G = %d \n\n", G);
      
      imgFinal[count] = G;

      aux = 0;
      auxC = 0;
      sumNumGx = 0;
      sumNumGy = 0;
      G = 0;
    }
    pakBaje +=m;
  }
}


int main(int argc, char ** argv){	
	char formato, numero;	//formato y numero se leen del archivo. 
	int m, n, protocolo; //m y n son las dimensiones de la imagen, protocolo se lee del archivo	
  int *img_rojo;
  int *img_verde;
  int *img_azul;
  FILE *imageFilePointer;
  
  imageFilePointer = fopen(argv[1], "r");

  if (imageFilePointer == NULL){
    printf("\n[!] ERROR: Algo salio mal al tratar de abrir el archivo %s\n", argv[1]);
    return 1;
  }

  printf("[!] Archivo %s abierto con exito!", argv[1]);

  //Se leen los dos primeros caracteres para luego imprimirlo en un nuevo archivo
  fscanf(imageFilePointer, "%c", &formato);
  fscanf(imageFilePointer, "%c", &numero);

  //Se capturan dimensiones de imagen y protocolo de colores
  fscanf(imageFilePointer, "%i", &m);
  fscanf(imageFilePointer, "%i", &n);
  fscanf(imageFilePointer, "%i", &protocolo);

  //Request de memoria para guardar imagen en memoria, separada por colores
  img_rojo  = (int*)malloc(m*n*sizeof(int));  if(img_rojo==NULL){     printf("Sin memoria (img_rojo) \n"); return 1; }
  img_verde = (int*)malloc(m*n*sizeof(int));  if(img_verde==NULL){     printf("Sin memoria (img_verde) \n"); return 1; }
  img_azul  = (int*)malloc(m*n*sizeof(int));  if(img_azul==NULL){     printf("Sin memoria (img_azul) \n"); return 1; }
  

  //Empieza la captura de colores por pixel en la imagen
  for (int i = 0; i < m * n; i++){
    fscanf(imageFilePointer, "%i ", &img_rojo[i]);
    fscanf(imageFilePointer, "%i ", &img_verde[i]);
    fscanf(imageFilePointer, "%i ", &img_azul[i]);
  }    
  fclose(imageFilePointer);

  // inicia el procesamiento de la imagen

   int Gx[] = {-1, 0, 1, -2, 0, 2, -1, 0, 1};
  int Gy[] = {1,2,1,0,0,0,-1,-2,-1};

  int *imgFinalR = (int*)malloc((m-2)*(n-2)*sizeof(int));
  int *imgFinalG = (int*)malloc((m-2)*(n-2)*sizeof(int));
  int *imgFinalB = (int*)malloc((m-2)*(n-2)*sizeof(int));

  convolucion(img_rojo, m, n, Gx, Gy, imgFinalR);
  convolucion(img_verde, m, n, Gx, Gy, imgFinalG);
  convolucion(img_azul, m, n, Gx, Gy, imgFinalB);
  generaImagen_salida(imgFinalR, imgFinalG, imgFinalB, (m-2), (n-2), protocolo, numero, formato, argv[2]);


  free(img_rojo);
  free(img_verde);
  free(img_azul);
  free(imgFinalR);
  free(imgFinalG);
  free(imgFinalB);
  return 0;

}

Writing ppm.c


In [5]:
%%script bash
gcc ppm.c -o ppm -lm
./ppm colores.ppm colores_sobel.ppm


[!] ERROR: Algo salio mal al tratar de abrir el archivo colores.ppm


CalledProcessError: ignored